## POMMA 数据下载模块
---
说明：模块提取POMMA指定目录的数据列表，生成一个名为“catalog.txt”的文件，里面每一行表示一个文件的下载链接，用户通过
```python
wget -i catalog.txt
```
完成下载。

### 1.Usage：./POMMA.py url [-o /path/to/your/ouput/file]

In [2]:
!./POMMA.py -h

usage: POMMA.py [-h] [-o O] url

positional arguments:
  url         文件目录的URL地址

optional arguments:
  -h, --help  show this help message and exit
  -o O        输出的文件路径


### 2.操作实例：解析pomma的某个分类目录catalog.html

In [4]:
!./POMMA.py http://opendap.bom.gov.au:8080/thredds/catalog/poama/realtime/monthly/m24a/emn/catalog.html

### 3.操作实例：输出结果文件到指定目录.eg:/home/xxx

In [4]:
!./POMMA.py http://opendap.bom.gov.au:8080/thredds/catalog/poama/realtime/monthly/m24a/emn/catalog.html -o /home/xxx

## ROC&BSS 检验模块
---
说明：针对三类数据（CSM106，POMMA，UMKO）进行检验，并产出相应图件。

### STEP 1.数据预处理
- CSM.py:
 处理中国气候中心的CSM106数据。
  1. 将指定目录下的所有数据读入；
  2. 基于各个月的月平均值据平；
  3. 导出NETCDF4 文件到指定目录；
  
- pomma.py:
 处理澳洲的POMMA数据数据。
  1. 将指定目录下的所有数据读入；
  2. 基于各个月的月平均值据平；
  3. 导出NETCDF4 文件到指定目录；

- UMKO.py:
 处理英国数据。
  1. 将指定目录下的所有数据读入；
  2. 基于各个月的月平均值据平；
  3. 导出NETCDF4 文件到指定目录；
  
---
### STEP 2.样本序列化
结合第一步得到的概率预报值，同时导入实际观测值，将网格上每个格点作为一个样本，
生成相应的键值对（真实值：三分类概率预报值），eg:
>y_true=[1,0,0] y_score=[0.15,0.5,0.35]

结果导出以CSV格式导出

- Serialization.py
 序列化样本操作。<br>
 1.读入观测数据和第一步得到的概率数据；<br>
 2.数据点匹配，计算格点概率；<br>
 3.序列化，以python pandas Dataframe表单保存数据；<br>
 4.结果以CSV或其他格式导出
  
---
### STEP 3.绘图
基于第二步得出的结果，可以依据需求绘制ROC曲线或BSS的网格图。
- ROC.py:
 绘制ROC曲线。
  1. 导入序列化样本文件；
  2. matplotlib 绘制图件并导出到指定目录；
  
- BSS.py:
 绘制BSS曲线。
  1. 导入序列化样本文件；
  2. matplotlib 绘制图件并导出到指定目录；


## 西太平洋副热带高压指数的预测模块
---
说明: 读入 500 hPa 位势高度和 u 方向风场数据文件, 计算给定年月的副热带高压指数. 
(TODO)如果给定的年月在数据中还没有发生, 则启动预测模块.

- 面积指数: 在 10N 以北 110E - 180 范围内, 500 hPa 位势高度场上所有位势高度不小于 588 dagpm 的格点围成的面积总和.
- 强度指数: 在 10N 以北 110E - 180 范围内, 500 hPa 位势高度场上所有位势高度不小于 588 dagpm 的格点围成的面积与该格点高度值减去 587 dagpm 差值的乘积的总和.
- 脊线指数: 在 10N 以北 110 - 150E 范围内, 588 dagpm 等值线所包围的副热带高压体内纬向风 u = 0 且 du/dy > 0 的特征线所在纬度位置的平均值; 若不存在 588 dagpm 等值线, 则定义 584 dagpm 等值线范围内纬向风 u = 0, 且 du/dy > 0 的特征线所在纬度位置的平均值; 若在某月不存在 584 dagpm 等值线, 则以该月的 1951 - 2010 年历史最小值代替.
- 西伸脊点: 在 90E - 180 范围内, 588 dagpm 最西格点所在的经度. 若在 90E 以西则统一记为 90E, 若在某月不存在 588 dagpm 等值线, 则以该月的 1951 - 2010 年历史最大值代替.

示例:
读入 hgt 文件以及 uwnd 文件, 指定时间 2014 年 10 月

In [ ]:
!./wpshi.py -hgt ~/Downloads/hgt.mon.mean.nc -uwnd ~/Downloads/uwnd.mon.mean.nc -y 2014 -m 10

输出:
(92, 173.0, 20.0, 100.0)

分别表示: 面积指数, 强度指数, 脊线指数, 西伸脊点